In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/challenge_cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Mahebourg,MU,2020-04-29 19:52:33,-20.41,57.70,73.40,83,75,10.29,Rain,0,0
1,1,Butaritari,KI,2020-04-29 19:52:33,3.07,172.79,82.44,81,100,12.64,Clouds,0,0
2,2,Rikitea,PF,2020-04-29 19:52:33,-23.12,-134.97,79.56,81,47,15.41,Rain,0,0
3,3,Clovis,US,2020-04-29 19:52:33,36.83,-119.70,89.01,33,75,5.82,Clouds,0,0
4,4,Cape Town,ZA,2020-04-29 19:52:33,-33.93,18.42,59.00,62,53,4.70,Clouds,0,0


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Cities that have rain.
rain_inches = city_data_df["Rain inches (last 3 hrs)"]
inches = []
for inch in rain_inches:
    inches.append(min(inch, 0))

In [7]:
# Cities that have snow.
snow_inches = city_data_df["Snow inches (last 3 hrs)"]
inches = []
for inch in snow_inches:
    inches.append(min(inch, 0))

In [8]:
# Create user prompts
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Mahebourg,MU,2020-04-29 19:52:33,-20.41,57.70,73.40,83,75,10.29,Rain,0,0
1,1,Butaritari,KI,2020-04-29 19:52:33,3.07,172.79,82.44,81,100,12.64,Clouds,0,0
2,2,Rikitea,PF,2020-04-29 19:52:33,-23.12,-134.97,79.56,81,47,15.41,Rain,0,0
3,3,Clovis,US,2020-04-29 19:52:33,36.83,-119.70,89.01,33,75,5.82,Clouds,0,0
4,4,Cape Town,ZA,2020-04-29 19:52:33,-33.93,18.42,59.00,62,53,4.70,Clouds,0,0
5,5,Albany,US,2020-04-29 19:52:06,42.60,-73.97,61.00,36,90,12.75,Clouds,0,0
7,7,Port Elizabeth,ZA,2020-04-29 19:50:06,-33.92,25.57,59.05,81,0,7.02,Clear,0,0
8,8,Quelimane,MZ,2020-04-29 19:52:34,-17.88,36.89,73.40,88,20,3.36,Clouds,0,0
10,10,Esperance,AU,2020-04-29 19:52:34,-33.87,121.90,53.60,71,90,8.01,Clouds,0,0
12,12,Mount Isa,AU,2020-04-29 19:52:35,-20.73,139.50,69.80,49,40,21.92,Clouds,0,0


In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,73.40,Rain,-20.41,57.70,
1,Butaritari,KI,82.44,Clouds,3.07,172.79,
2,Rikitea,PF,79.56,Rain,-23.12,-134.97,
3,Clovis,US,89.01,Clouds,36.83,-119.70,
4,Cape Town,ZA,59.00,Clouds,-33.93,18.42,
...,...,...,...,...,...,...,...
551,Irece,BR,76.60,Clouds,-11.30,-41.86,
552,Tibati,CM,73.13,Rain,6.47,12.63,
554,El Vigia,VE,86.38,Rain,8.62,-71.65,
555,Amparafaravola,MG,60.37,Clouds,-17.58,48.22,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

In [16]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a map of the locations
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,73.40,Rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
1,Butaritari,KI,82.44,Clouds,3.07,172.79,Isles Sunset Lodge
2,Rikitea,PF,79.56,Rain,-23.12,-134.97,Pension Maro'i
3,Clovis,US,89.01,Clouds,36.83,-119.70,Homewood Suites by Hilton Fresno Airport/Clovi...
4,Cape Town,ZA,59.00,Clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
...,...,...,...,...,...,...,...
551,Irece,BR,76.60,Clouds,-11.30,-41.86,RADAR INN HOTEL
552,Tibati,CM,73.13,Rain,6.47,12.63,Tibati Hotel
554,El Vigia,VE,86.38,Rain,8.62,-71.65,Liceo Bolivariano 19 De Abril
555,Amparafaravola,MG,60.37,Clouds,-17.58,48.22,Aina Franco


In [20]:
# Create the output file (CSV).
output_data_file = "weather_data/vacation_challenge.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")